In [1]:
import pandas as pd
import numpy as np

In [2]:
# --- 步驟 1: 設定檔案路徑與所需欄位 ---
original_file_path = '113年數位近用調查/data113_1.csv' 
output_file_path = 'working_population_health_data.csv'

# 定義需要從資料集中抽取的欄位名稱
columns_to_keep = [
    # 分組/調節變數
    'v4',   # 年齡
    'v45',  # 性別
    
    # 自變數 (數位工作壓力)
    'v24',  # 是否遠距工作
    'v41',  # 工作內容使用網路百分比
    'v43',  # 下班後是否處理工作
    'v42',  # 被AI取代的焦慮感

    # 依變數 (身心健康指標)
    'v33',  # 身體狀況是否變差
    'v34',  # 主觀幸福感

    # 控制變數 & 篩選變數
    'v8',  # 每周上網頻率 (新加入)
    'v38',  # 教育程度
    'v39'   # 行業 (用於篩選在職者)
]

In [3]:
# --- 步驟 2: 讀取與處理資料 ---

print(f"正在從 '{original_file_path}' 讀取資料...")
df_full = pd.read_csv(original_file_path)
print("原始資料集讀取成功！")

# (可選) 如果您不確定欄位名稱，可以取消下面這行的註解來查看所有欄位
# print("所有欄位名稱:", df_full.columns.tolist())

# 從完整資料中，只選取需要的欄位
print(f"正在選取研究需要的欄位: {columns_to_keep}")
df_selected = df_full[columns_to_keep].copy() # 使用 .copy() 避免 SettingWithCopyWarning

正在從 '113年數位近用調查/data113_1.csv' 讀取資料...
原始資料集讀取成功！
正在選取研究需要的欄位: ['v4', 'v45', 'v24', 'v41', 'v43', 'v42', 'v33', 'v34', 'v8', 'v38', 'v39']


In [4]:
# --- 步驟 3: 篩選研究對象 (在職者) ---
# 根據主調查問卷Q39的選項，非在職者選項為 (20)家管, (21)學生, 
# (22)在找工作/等待恢復工作, (23)退休，將這些排除。
# 97, 98 通常代表其他或拒答，也一併排除。
non_working_codes = [20, 21, 22, 23, 97, 98]

original_rows = len(df_selected)
print(f"篩選前總樣本數: {original_rows}")

# 執行篩選
df_working_population = df_selected[~df_selected['v39'].isin(non_working_codes)]

filtered_rows = len(df_working_population)
print(f"篩選後在職者樣本數: {filtered_rows} (移除了 {original_rows - filtered_rows} 筆非在職樣本)")

篩選前總樣本數: 15133
篩選後在職者樣本數: 7461 (移除了 7672 筆非在職樣本)


In [ ]:
# --- 步驟 4: 處理「不知道/拒答」的遺失值---
# 定義所有代表「不知道/拒答」的編碼，根據問卷，大部分是 98。 Q42 選項有(98)不知道/拒答，Q34 也有 (98)。
# 設定一個通用列表
missing_value_codes = [97, 98, 99, 998] # 涵蓋常見的拒答碼

# 將所有欄位中的這些特殊代碼，替換成 pandas 標準的缺失值 np.nan
# 這一步不會刪除任何資料，只是做一個標記
for col in df_working_population.columns:
    df_working_population[col] = df_working_population[col].replace(missing_value_codes, np.nan)

# 計算替換後的缺失值數量
missing_before_drop = df_working_population.isnull().sum().sum()
print(f"標記出 {missing_before_drop} 個 '不知道/拒答' 的儲存格。")

# 執行「列刪除法」，刪除任何一個欄位含有 np.nan 的整筆資料
original_rows = len(df_working_population)
df_cleaned = df_working_population.dropna()
final_rows = len(df_cleaned)

print(f"因 '不知道/拒答' 而移除 {original_rows - final_rows} 筆資料。")
print(f"最終用於分析的樣本數為: {final_rows}")

標記出 1919 個 '不知道/拒答' 的儲存格。
因 '不知道/拒答' 而移除 797 筆資料。
最終用於分析的樣本數為: 6664


C:\Users\USER\AppData\Local\Temp\ipykernel_17620\1840732860.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_working_population[col] = df_working_population[col].replace(missing_value_codes, np.nan)


In [7]:
# --- 步驟 5: 儲存最終乾淨的資料 ---
df_cleaned.to_csv(output_file_path, index=False, encoding='utf-8-sig')

print("-" * 50)
print(f"資料集已儲存至:\n{output_file_path}")
print("-" * 50)
print("\n前5筆乾淨資料預覽:")
print(df_cleaned.head())


--------------------------------------------------
資料集已儲存至:
working_population_health_data.csv
--------------------------------------------------

前5筆乾淨資料預覽:
    v4  v45  v24  v41  v43  v42  v33  v34   v8  v38  v39
82   7    1  2.0  0.0  2.0  2.0  2.0  8.0  4.0  4.0    1
83   6    2  2.0  0.0  2.0  3.0  3.0  7.0  2.0  3.0    1
84   8    1  2.0  0.0  2.0  3.0  3.0  6.0  2.0  3.0    1
85   8    1  2.0  0.0  2.0  2.0  3.0  8.0  5.0  3.0    1
86   7    1  2.0  0.0  2.0  3.0  3.0  6.0  3.0  5.0    3
